# PyUAT: Tracking example

In this example we demonstrate the `PyUAT` tracking method tracking a sequence of the [One-in-a-million](https://doi.org/10.5281/zenodo.7260137) dataset. You can perform the tracking with the different configurations presented in our paper.

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.7260137.svg)](https://doi.org/10.5281/zenodo.7260137)

## 1. Install software dependencies

Install the required software packages

In [ ]:
%pip install gurobipy moviepy
%pip install uatrack==0.0.2

## 2. Download image data and segmentation

We download the imaging data and the ground truth segmentation & tracking data 🚀

In [ ]:
!wget -O filtered_0.json https://fz-juelich.sciebo.de/s/5vBB6tW8c2DpaU3/download
!wget -O 00_stack.tif https://fz-juelich.sciebo.de/s/Xge7fj56QM5ev7q/download

# 3. Prepare the tracking configuration

In order to track cell using `PyUAT`, we need to define models that score the properties of assignments. That is performed in the section below and results in the construction of assignment generators that generate and score the different assignment types.

In [1]:
subsampling_factor = 2
end_frame = 400

# activate one tracking configuration (uncomment any other tracking configuration from the paper)
#tracking_configuration = "NN"
tracking_configuration = "FO"
#tracking_configuration = "FO+G"
#tracking_configuration = "FO+O"
#tracking_configuration = "FO+DD"
#tracking_configuration = "FO+G+O+DD"

# Load segmentation data

In [2]:
# TODO load data

from uatrack.utils import load_data

image_file = "00_stack.tif"
tracking_file = "filtered_0.json"

overlay, tracking_graph = load_data(tracking_file, subsampling_factor=subsampling_factor, end_frame=end_frame)

/home/johannes/miniconda3/envs/uat-examples/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-10 15:34:28,007	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
findfont: Font family ['DejaVuSans'] not found. Falling back to DejaVu Sans.


In [3]:
print("Total number of cell detections: ", len(overlay))

Total number of cell detections:  3360


## 4. Perform the tracking

In [4]:
import time

from uatrack.utils import extract_single_cell_information, save_tracking
from uatrack.core import simpleTracking
from uatrack.config import setup_assignment_generators

input_file = "segmentation.json"
output_file = "simpleTracking.json.gz"

# Fix arguments
num_particles = 1 
num_cores = 1
max_num_hypotheses = 1
cutOff = -1
max_num_solutions = 1

print("Extract single-cell information...")
df, all_detections = extract_single_cell_information(overlay)

print("Setup assignment generators...")
assignment_generators = setup_assignment_generators(df, subsampling_factor, tracking_configuration)

print("Perform tracking...")
# start tracking
start = time.time()
res = simpleTracking(
    df,
    assignment_generators,
    num_particles,
    num_cores=num_cores,
    max_num_hypotheses=max_num_hypotheses,
    cutOff=cutOff,
    max_num_solutions=max_num_solutions,
    mip_method="CBC",  # use "GRB" if you have gurobi installed
)
end = time.time()

print("time for tracking", end - start)

save_tracking(res[0], all_detections, output_file)


Extract single-cell information...
Setup assignment generators...
Perform tracking...


Perform tracking: 100%|██████████| 200/200 [01:28<00:00,  2.25it/s]


time for tracking 88.72669720649719


## 4. Render tracking video

In order to visually assess the quality of the tracking, we can render it into a video. We use the `acia` library for that.

In [5]:
from acia.tracking.formats import parse_simple_tracking
import gzip

# read the tracking result
with gzip.open("simpleTracking.json.gz") as input_file:
  overlay, pred_tracking_graph = parse_simple_tracking(input_file.read())

In [6]:
import tifffile
import numpy as np

from acia.segm.local import InMemorySequenceSource
from acia.tracking import subsample_tracking, TrackingSourceInMemory
from acia.viz import render_tracking, render_segmentation, render_video


ts = TrackingSourceInMemory(overlay, pred_tracking_graph)

image_stack = tifffile.imread(image_file)
raw_images = InMemorySequenceSource(np.repeat(image_stack[:end_frame][::subsampling_factor,...,None], repeats=3, axis=-1))

segm_images = render_segmentation(raw_images, overlay)
tracked_images = render_tracking(segm_images, overlay, pred_tracking_graph)

render_video(tracked_images, "tracked.mp4", 10, "vp9")

200it [00:01, 194.14it/s]
200it [00:00, 217.45it/s]


Moviepy - Building video /home/johannes/projects/work/uat-new/tracked.mp4.
Moviepy - Writing video /home/johannes/projects/work/uat-new/tracked.mp4



Moviepy - Done !
Moviepy - video ready /home/johannes/projects/work/uat-new/tracked.mp4


# Show the video

In [7]:
# embed the video in the notebook
import moviepy.editor
moviepy.editor.ipython_display("tracked.mp4", maxduration=300)